## Find Missing index raster values based on hru centroid

Use Gdal and geopandas to find index raster cell values for hrus that are much smaller than 1 grid cell.

Theodore Barnhart | tbarnhart@usgs.gov

In [1]:
import gdal
import geopandas as gpd

In [2]:
def get_cell(df,dat=[],gt=[],rb=[]):
    '''Input a missing hru data frame'''
    regHRUid = df.hruID # pull the reg
    geom = dat.loc[dat.hru_id_reg == regHRUid].geometry
    mx = geom.exterior.centroid.x
    my = geom.exterior.centroid.y
    px = int((mx-gt[0])/gt[1]) # x pixle
    py = int((my-gt[3])/gt[5]) # y pixle
    intval = rb.ReadAsArray(px,py,1,1)
    return intval[0][0] 

In [3]:
def find_missing(reg,infile='tmp'):
    '''
    In:
    reg = region to work with
    infile = index raster to pick values from based on hru centroid
    
    Out:
    saves a missing data pickle
    '''
    
    missing = pd.read_pickle('./data/missing_region_%s.pcl'%reg) # load the missing filed data frame
    fl = './data/nhrus/clean_AEA/nhru_%s_clean.shp'%reg
    dat = gpd.read_file(fl) # load the correct shapefile
    
    # load the index raster
    src_ds = gdal.Open(infile)
    gt = src_ds.GetGeoTransform()
    rb = src_ds.GetRasterBand(1)
    
    missing['idx'] = missing.apply(get_cell,dat=dat,gt=gt,rb=rb,axis=1) # apply the finder function
    
    # code to convert the output to a missing hru contributions data frame
    # save the missing hru contributions data frame
    
    return missing

In [4]:
reg = '04'
missing = find_missing(reg,infile='./data/hrap_grid_AEA.tiff')

In [5]:
missing.head()

,hruID,cutline,region,inpath,outpath,isfile,out,cols,rows,cells,idx
116,117,/home/tbarnhart/projects/NHM_precipitation/dat...,04,./hrap_grid_AEA_idx_sm.img,./AEA_tiffs/HUC_04_hruID_117.tiff,False,CompletedProcess(args='gdalwarp -cutline /home...,0,1,0,880
3746,3747,/home/tbarnhart/projects/NHM_precipitation/dat...,04,./hrap_grid_AEA_idx_sm.img,./AEA_tiffs/HUC_04_hruID_3747.tiff,False,CompletedProcess(args='gdalwarp -cutline /home...,0,0,0,753
3835,3836,/home/tbarnhart/projects/NHM_precipitation/dat...,04,./hrap_grid_AEA_idx_sm.img,./AEA_tiffs/HUC_04_hruID_3836.tiff,False,CompletedProcess(args='gdalwarp -cutline /home...,1,0,0,720
4538,4539,/home/tbarnhart/projects/NHM_precipitation/dat...,04,./hrap_grid_AEA_idx_sm.img,./AEA_tiffs/HUC_04_hruID_4539.tiff,False,CompletedProcess(args='gdalwarp -cutline /home...,0,0,0,706
5067,5068,/home/tbarnhart/projects/NHM_precipitation/dat...,04,./hrap_grid_AEA_idx_sm.img,./AEA_tiffs/HUC_04_hruID_5068.tiff,False,CompletedProcess(args='gdalwarp -cutline /home...,1,0,0,735
